# Python程序设计与数据科学导论 期中大作业
## 动态时间序列分析与预测
负责人：李隽仁 姜腾

作业发布时间：2022.5.1

**作业截止时间：2022.5.15 23：59（若采用深度学习模型，最晚可以延后至5.19 23：59，不影响成绩评定，但需在5.8前向助教申请）**

提交内容：zip格式，内含源代码及pdf版本的大作业报告。
### 概述
本次大作业在HW4的交易所架构上完善而来，目标依然为**从动态的交易中赚到尽量多的钱**，请回顾HW4中的相关环境。

为了填补上次作业中的漏洞，我们做了以下补充：
- 完善了撮合交易系统中的**时间优先**原则，先提交的订单会被优先响应（此处致谢杜雨时同学在HW4中的优秀作业）；
- 在提交订单/撮合交易前会检查**账户资金/持有股数**是否能够满足交易进行，否则将作为无效挂单处理；
- 对交易征收**佣金**，防止大量挂单自买自卖；
- 为了符合现实交易环境，每天的挂单在**闭市后若无成功交易则会被清除**；
- 最终比较的标准为**持有现金数**，而非总资产。


### 环境介绍
本次大作业设定为美股交易市场，有以下基础信息：
- 交易所共有A: Twitter, Inc. (TWTR)、B: The Coca-Cola Company (KO)、C: General Motors Company (GM)三只股票，时间窗口为2013.11.7-2022.4.29，价格走向由机器人维持，和历史上的真实股价趋势保持一致。为减少机器人的数目，我们把三只股票的股价都做了十分之一处理。
- 三只股票虽然所属行业不同，但在有的时段会表现出相同的价格趋势（例如2020年初的暴跌），正确地把握三只股票间的关系会对提升盈利有所帮助。
- Bot分三类：随机买卖\*14、贪心买卖\*6、神\*6；前两类Bot你已经在HW4打过交道，最后一类Bot可以看到股票的历史价格，从而维持股价和历史趋势接近。
- Bot的初始资金为1,000,000美元，初始持有每只股票100,000股，足以保证在交易过程中不触发资金不足&股票不足的问题；**你的初始资金为10,000美元，初始持有每只股票1,000股**，因此请注意，像HW4附加题中**无限量买入/抛售的做法在这里将不再奏效**，在积累足够资本之前想操纵股票的走向也并没有那么容易。
- 虽然系统后台Bot会根据历史的股票的价格驱动股价的大体走势，但在作为交易者选择交易策略时这个信息理论上并不可见，在设计算法及策略时**不要予以参考**，所有以任何形式参考该信息的会被认定为错误方案，最高只会获得70分的成绩。你能够参考的信息包括：从开始以来的历史股价，交易所中的历史订单，此刻场上现有的挂单等等。
- 每一天你只可以**入场一次**，进行**任意次买/卖挂单**，每一单默认为100股，佣金2美元。26个Bot的入场顺序会被随机打乱，但**你可以选择自己入场的时间**（这也是策略的一部分）。提示：到得太晚可能会错过这一天里的机遇，到得太早则会面临这一天中市场的未知风险。
- 美股实行T+0制度，你可以同时挂买单和卖单，但要想好这么做的意义——否则会白白浪费佣金。

特别的，由于系统代码环境开发时间较紧，若存在**漏洞和不完备的地方**（e.g.像HW4中用非常简单的策略赢得大量资产）欢迎同学指出，我们也会在第一时间更新完善。相关漏洞的发现举报会适当考虑作为加分项，但直接利用漏洞取得超值效益不会对成绩有正面影响。

### 任务及评分标准
1. 用课程所学的时间序列知识(20')动态分析预测股价，可以结合任一种/多种机器学习模型(20')，选择合适的时间策略&定价策略买入卖出，为自己争取最大化盈利(10')。
2. 撰写期中大作业报告，内容至少应当包含：你用到的时间序列技术和机器学习模型(30')，你对三只股票的操作策略分析（若策略有不同之处请分析差异的来源）(20')，你最终的现金；在完成大作业的过程中如果有多种尝试或其他感想，鼓励一并写入。
3. 加分项：1）大作业报告完整清晰，阐述合理的交易策略和解决方案，有交流报告价值(+1'\~+10')；2）在所有作业提交后，我们会综合大家的策略在一起进行比赛，前百分之五十将按比例（向上取整）均匀获得+5'~+1'的附加分数；所有加分项合并不超过10分。

再次提醒注意：若预测过程中用到了Stock_Price.csv中的历史股价数据，期中大作业最高分为70分⚠。

# Have Fun!

In [ ]:
from copy import deepcopy
from abc import abstractmethod
import time

class MinHeap: # 最小堆的代码已给出，以订单的价格进行排序；请注意这里存储的是订单而不仅是订单的价格
    def __init__(self, stock_name):
        self.heap_list = [BuyStockOrder(stock, '', stock_name, 0)]
        self.current_size = 0

    def insert(self, k):
        self.heap_list.append(k)
        self.current_size+=1
        size=self.current_size
        self.sift_up(size)

    def sift_up(self, i):
        while i // 2 > 0:
            if self.heap_list[i].price < self.heap_list[i // 2].price:
                self.heap_list[i], self.heap_list[i // 2] = self.heap_list[i // 2], self.heap_list[i]
            elif self.heap_list[i].price == self.heap_list[i // 2].price:
                # 如果价格相同的情况, 时间小的上移
                if self.heap_list[i].t < self.heap_list[i // 2].t:
                    self.heap_list[i], self.heap_list[i // 2] = self.heap_list[i // 2], self.heap_list[i]
            i = i // 2

    def sift_down(self, i):
        while (i * 2) <= self.current_size:
            mc = self.min_child(i)
            if self.heap_list[i].price > self.heap_list[mc].price:
                self.heap_list[i], self.heap_list[mc] = self.heap_list[mc], self.heap_list[i]
            elif self.heap_list[i].price == self.heap_list[mc].price:
                # 如果价格相同的情况, 时间小的上移
                if self.heap_list[i].t > self.heap_list[mc].t:
                    self.heap_list[i], self.heap_list[mc] = self.heap_list[mc], self.heap_list[i]
            i = mc

    def min_child(self, i):
        if (i * 2) + 1 > self.current_size:
            return i * 2
        else:
            if self.heap_list[i * 2].price < self.heap_list[(i * 2) + 1].price:
                return i * 2
            else:
                # 价格相同，时间小的小
                if self.heap_list[i * 2].price == self.heap_list[(i * 2) + 1].price:
                    if self.heap_list[i * 2].t < self.heap_list[(i * 2) + 1].t:
                        return i * 2
                return (i * 2) + 1

    def delete_min(self):
        if len(self.heap_list) == 1:
            return 'Empty heap'

        root = self.heap_list[1].price
        self.heap_list[1].price = self.heap_list[self.current_size].price
        *self.heap_list, _ = self.heap_list
        self.current_size -= 1

        self.sift_down(1)
        return root
    
    def sort(self):
        ls=[]
        while(len(self.heap_list)>1):
            if self.heap_list[1].price > 0:
                ls.append(f'{self.heap_list[1].price}')
            else:
                ls.append(f'{-self.heap_list[1].price}') 
            self.delete_min()
        return ls
    

class Order:
    @abstractmethod
    def execute():
        pass
    
class BuyStockOrder(Order):
    def __init__(self, stock, name, stock_name, price):
        self.stock = stock
        self.name = name
        self.price = price
        self.t = time.time()
        self.stock_name = stock_name
        self.order_type = 'Buy'
        
    def execute(self):
        self.stock.buy(self.name, self.stock_name, self.t, self.price)
        
class SellStockOrder(Order):
    def __init__(self, stock, name, stock_name, price):
        self.stock = stock
        self.name = name
        self.price = price
        self.t = time.time()
        self.stock_name = stock_name
        self.order_type = 'Sell'
        
    def execute(self):
        self.stock.sell(self.name, self.stock_name, self.t, self.price)
        
class StockTrade:
    _instance = None # 启用单例模式
    def __new__(cls):
        if cls._instance is None:
            cls._instance = object.__new__(cls)
        return cls._instance
    def __init__(self):
        self.market = {}
        self.money = {}
        
    def buy(self, name, stock_name, t, price):
        self.price = price
        #if name =='You':
            #print(f'{name} bought stock {stock_name} at {price}') # 注释避免冗余输出刷屏，如果需要检查具体交易过程也可以打印出来
        self.market[f'{name}_{stock_name}'] += 100
        self.money[name] -= 100 * price
        self.money[name] -= 0.02 * 100 # 交易佣金，在A股按交易金额比例收取，美股按交易股数定额收取，此处收取0.02美分/股
        
    def sell(self, name, stock_name, t, price):
        self.price = price
        #if name =='You':
            #print(f'{name} sold stock {stock_name} at {price}')
        self.market[f'{name}_{stock_name}'] -= 100
        self.money[name] += 100 * price
        self.money[name] -= 0.02 * 100 # 交易佣金，同上
    
    def __str__(self):
        return '\n'.join([f'{k}: {v}' for k, v in zip(self.market.keys(), self.market.values())])
    
    def show_money(self):
        return '\n'.join([f'{k}: {v}' for k, v in zip(self.money.keys(), self.money.values())])
    
class Agent:
    def __init__(self, stock):
        self.stock = stock
        self.__OrderQueue = []
        self.__BuyQueue = [MinHeap('TWTR'), MinHeap('KO'), MinHeap('GM')] # 三只股票对应三个最小堆
        self.__SellQueue = [MinHeap('TWTR'), MinHeap('KO'), MinHeap('GM')]
        self.price = [[4.49], [3.04], [2.78]] # 三只股票起始股价
        
    def placeOrder(self, order):
        if order.name not in self.stock.money.keys(): # 为每个交易者初始化，这里对Bot和对You是区别对待的:D
            self.stock.money[order.name] = 1e6 if order.name != 'You' else 1e4
            for s_name in ['TWTR', 'KO', 'GM']:
                self.stock.market[f'{order.name}_{s_name}'] = int(1e5) if order.name != 'You' else int(1e3)
        order.price = round(order.price, 2)
        stock_ID = ['TWTR', 'KO', 'GM'].index(order.stock_name)
        # 限制资金下限：不能白嫖股票
        if order.order_type == 'Buy' and 100 * order.price > self.stock.money[order.name] and order.name == 'You':
            print(f'Invalid Order: {order.name} don\'t have enough money to buy stock {order.stock_name} at {order.price}.')
        # 限制股票下限：不能卖自己都没有的股票
        elif order.order_type == 'Sell' and self.stock.market[f'{order.name}_{order.stock_name}'] <= 0 and order.name == 'You':
            print(f'Invalid Order: {order.name} don\'t have enough shares of stock {order.stock_name} to sell.')
        else:
            self.__OrderQueue.append(deepcopy(order))
            if order.order_type == 'Buy':
                order.price = -order.price
                self.__BuyQueue[stock_ID].insert(order)
            else:
                self.__SellQueue[stock_ID].insert(order)
            if len(self.__BuyQueue[stock_ID].heap_list) > 1 and len(self.__SellQueue[stock_ID].heap_list) > 1:
                # 同上，做了资金和股票数的限制
                if -self.__BuyQueue[stock_ID].heap_list[1].price >= self.__SellQueue[stock_ID].heap_list[1].price and stock.market[f'{self.__SellQueue[stock_ID].heap_list[1].name}_{self.__SellQueue[stock_ID].heap_list[1].stock_name}'] > 0 and stock.money[f'{self.__BuyQueue[stock_ID].heap_list[1].name}'] >= 100 * self.__BuyQueue[stock_ID].heap_list[1].price:
                    buy_order = self.__BuyQueue[stock_ID].heap_list[1]
                    sell_order = self.__SellQueue[stock_ID].heap_list[1]
                    buy_order.price = -buy_order.price
                    curr_price = buy_order.price if buy_order.price < self.price[stock_ID][-1] else (sell_order.price if sell_order.price > self.price[stock_ID][-1] else self.price[stock_ID][-1])
                    buy_order.price = curr_price
                    sell_order.price = curr_price
                    buy_order.execute()
                    sell_order.execute()
                    self.__BuyQueue[stock_ID].delete_min()
                    self.__SellQueue[stock_ID].delete_min()
                    self.price[stock_ID].append(curr_price)
                
    def show_buy(self):
        queues = deepcopy(self.__BuyQueue)
        output = []
        for i, queue in enumerate(queues):
            output.append((f'Stock {chr(65+i)}:'))
            output.append('\n'.join(deepcopy(queue).sort()))
        return '\n'.join(output)
    
    def show_sell(self):
        queues = deepcopy(self.__SellQueue)
        output = []
        for i, queue in enumerate(queues):
            output.append((f'Stock {chr(65+i)}:'))
            output.append('\n'.join(deepcopy(queue).sort()))
        return '\n'.join(output)
        
    def history_price(self):
        return self.price
    
    def close_market(self): # 每天闭市时重新初始化挂单列表，清空此前的挂单
        self.__BuyQueue = [MinHeap('TWTR'), MinHeap('KO'), MinHeap('GM')]
        self.__SellQueue = [MinHeap('TWTR'), MinHeap('KO'), MinHeap('GM')]
        return
    
    def __str__(self):
        return '\n'.join([f'{order.name}: Tried to {order.order_type} stock {order.stock_name} at {order.price}' for order in self.__OrderQueue])

In [ ]:
stock = StockTrade()
agent = Agent(stock)
# 让Alice，Bob和Carol试着交易一些股票
agent.placeOrder(BuyStockOrder(stock, 'Alice', 'TWTR', 4.49))
agent.placeOrder(BuyStockOrder(stock, 'Bob', 'GM', 2.78))
agent.placeOrder(SellStockOrder(stock, 'Bob', 'KO', 3.03))
agent.placeOrder(SellStockOrder(stock, 'Alice', 'GM', 2.77))
agent.placeOrder(BuyStockOrder(stock, 'Carol', 'TWTR', 4.49))
agent.placeOrder(SellStockOrder(stock, 'Carol', 'TWTR', 4.50))
agent.placeOrder(BuyStockOrder(stock, 'You', 'KO', 3.04))
# 依次打印交易情况，持股情况和资金情况
print('\nHistory Order', agent, sep='\n')
print('\nCurrent Stock', stock, sep='\n')
print('\nRemaining Buy Order', agent.show_buy(), sep='\n')
print('\nRemaining Sell Order', agent.show_sell(), sep='\n')
print('\nHistory Price', agent.history_price(), sep='\n')
print('\nCurrency', stock.show_money(), sep='\n')

**请补完代码，实现对数据的分析和自己的策略，用初始的10000美元赚到尽可能多的钱。**


In [ ]:

   

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from random import randint, random, shuffle
from tqdm import trange
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
# 主要策略前期（400交易以前）使用LR回归预测买卖点，后期（400以后）使用kalman处理后arma预测并根据短线趋势判断
def func(_history_price): 
    #-1:sell, 1:buy
    history_price=pd.DataFrame(_history_price)
    if (len(history_price)<=400):
        return 0
    else:
        #两个算法（或者如果要进一步完成可以引入更多）权重
        vote=0
        weight_arma=5
        weight_lr=1
        history_price=pd.DataFrame(_history_price)
        Y=np.array(history_price[:])
        '''
        #history_price=pd.DataFrame(history_price).diff(periods=7)[7:] #不收敛
        history_price=pd.DataFrame(history_price)
        Y=np.array(history_price[4:])
        data_array=[[i] for i in Y ]
        array_data=[i for i in Y ]
        '''
        data_array=[[i] for item in Y for i in item]
        array_data=[i for item in Y for i in item]
        length=len(array_data)
        #print(length)
        max_len=300
        tlen=5
        tshe=7
        #直接通过定义极值得到标签集（专家系统）,极值的问题是需要两侧数据，但是实际上只有单侧，所以要学习单侧的特征并预测
        #less_peak=signal.argrelextrema(Y,np.less,order=3)
        #greater_peak=signal.argrelextrema(Y,np.greater,order=3)
    
        less_peak=[]
        for i in range(length-max_len,length-tlen):
            count=0
            for j in range(i-tlen,i+tlen):
                if array_data[i]<array_data[j]:
                    count+=1
            if count>=tshe:
                less_peak.append(i)   
        greater_peak=[]
        for i in range(length-max_len,length-tlen):
            count=0
            for j in range(i-tlen,i+tlen):
                if array_data[i]>array_data[j]:
                    count+=1
            if count>=tshe:
                greater_peak.append(i)
            
        '''
        less_peak=signal.argrelextrema(Y,np.less,order=3)
        greater_peak=signal.argrelextrema(Y,np.greater,order=3)
        '''

        #线性回归或者SVM扩增序列特征
        #先试试线性回归
        clf_buy=None
        clf_sell=None
    
        def linearReg(size):
            tmp=[]
            for i in range(length-max_len,length):
                a=[j for j in range(0,size)]
                b=data_array[i-size:i]
                c=LinearRegression().fit(b,a).coef_
                tmp.append(list(c))#啊啊要用list转出来，不然没法变成数组
            tmp=[k for item in tmp for k in item]
            return tmp
        #print(LinearRegression().fit(X,Y).coef_)
        linear_3=linearReg(3)
        #print(linear_3)
        linear_5=linearReg(5)
        linear_10=linearReg(10)
        linear_20=linearReg(20)
        total_feature=[]
        for i in range(0,max_len):
            tmp=[]
            for j in range(0,6):
                tmp.append(array_data[i+length-max_len-j])
            tmp.append(linear_3[i])
            tmp.append(linear_5[i])
            tmp.append(linear_10[i])
            tmp.append(linear_20[i])
            tmp=np.array(tmp)
            total_feature.append(tmp) 
        total_feature=np.array(total_feature)
        #print(total_feature)
        def MaxMinNormalization(a):
            Min=np.min(a)
            Max=np.max(a)
            b=[]
            for t in a:
                t = (t - Min) / (Max - Min)
                b.append(t)
            return b
        #print(total_feature)
        for i in range(0,10):
            total_feature[:,i]=MaxMinNormalization(total_feature[:,i])
            total_feature[:,i]=np.array(pd.Series(total_feature[:,i]).fillna(value=0))
            
        #print(total_feature)
        #total_feature.append([array_data[i+20],linear_3[i],linear_5[i]])
        #print(train_feature)
        #制作标签集
        #total_Y=Y[length-max_len:]
        total_less_labels=np.ones(length)    #极大点标签为1  默认什么都不是标记为1
        for i in less_peak:
        #for i in less_peak:
            if i>10 and i<length:
                total_less_labels[i]=0 #极小点标签为0
        total_less_labels[length-max_len+1]=0
        total_greater_labels=np.ones(length)    #极大点标签为1
        for i in greater_peak:
        #for i in greater_peak:
            if i>10 and i<length:
                total_greater_labels[i]=0 #极小点标签为0
        total_greater_labels[length-max_len+1]=0

        #train_feature,test_feature,train_less_labels,test_labels=train_test_split(total_feature,total_less_labels)
        train_t1=0
        train_t2=max_len-1
        test_t1=length-1
        test_t2=1800
        train_feature=total_feature[0:max_len-1]
        #print(len(train_feature))
        train_less_labels=total_less_labels[length-max_len:length-1]
        #test_less_labels=total_less_labels[test_t1:test_t2]
        train_greater_labels=total_greater_labels[length-max_len:length-1]

        #test_greater_labels=total_less_labels[test_t1:test_t2]
        #buy_index=[i for i in range(length-max_len,length) if total_greater_labels[i]==0]
        '''
        sell_index1=[i for i in range(length-max_len,length) if total_less_labels[i]==0]
        sell_index=[i+length-max_len for i in range(length-max_len,length) if total_less_labels[i]==0]
        plt.figure(figsize=(16,4))
        plt.plot(np.arange(max_len),Y[length-max_len:length])
        plt.plot(sell_index1,Y[sell_index],'+')
        '''
        sell_index1=[i for i in range(0,len(train_less_labels)) if train_less_labels[i]==0]
        sell_index=[i+length-max_len for i in range(0,len(train_less_labels)) if train_less_labels[i]==0]
        '''
        if length<=500:
            plt.figure(figsize=(16,4))
            plt.plot(np.arange(max_len),Y[length-max_len:length])
        #plt.plot(less_peak[0],Y[less_peak[0]],'o')
            plt.plot(sell_index1,Y[sell_index],'o')
        '''
        clf_buy = LogisticRegression(class_weight={0:0.85,1:0.15}).fit(train_feature,train_less_labels)  #建立预测极值点模型 ,只用一维肯定是不靠谱的，只是根据绝对值取分类，必须特征维数序列扩增
        clf_sell=LogisticRegression(class_weight={0:0.85,1:0.15}).fit(train_feature,train_greater_labels)
        a=clf_buy.predict(train_feature)
        buy_index1=[i for i in range(0,len(a)) if a[i]==0]
        buy_index=[i+length-max_len for i in range(0,len(a)) if a[i]==0]
        '''
        if length<=500:
            plt.figure(figsize=(16,4))
            plt.plot(np.arange(max_len),Y[length-max_len:length])
            plt.plot(buy_index1,Y[buy_index],'o')
        plt.plot(sell_index,Y[sell_index],'+')
        '''
        test_feature=[total_feature[max_len-1]]
        buy_point=clf_buy.predict(test_feature)
        buy_point=[int(i)for i in buy_point]

        if buy_point[0]==0:
            vote+=-1*weight_lr
            
        sell_point=clf_sell.predict(test_feature)
        sell_point=[int(i)for i in sell_point]
        if sell_point[0]==0:
            vote+=1*weight_lr

        for i in range(0,1):
            data=history_price.diff(periods=7) 
        data=data[7:]
            #print(adfuller(data[7:]))
        def Kalman_filter(x,P0=0.02,q=0.02,r=0.55):
	        # 状态预测
            x_filter = x.copy()
            p_last = P0
            for i in range(1,x.shape[0]):
    	        # 协方差预测公式
                p_mid = p_last + q
                # 卡尔曼系数
                k = p_mid / (p_mid + r)
                # 状态估计
                x_filter[i] = x_filter[i-1] + k * (x[i] - x_filter[i-1])
                # 噪声协方差更新
                p_last = (1 - k) * p_mid
            return x_filter
        data=Kalman_filter(np.array(data))
        data=pd.DataFrame(data)
        #data=data[7::4]
        length=len(data)
        #print(length)
        #print(data[length-1])
        max_len=100
        tempModel = sm.tsa.arima.ARIMA(data[length-max_len:length-1],order=(7,0,4),enforce_stationarity=False).fit()
        '''
        tmp=[i for item in np.array(data[length-20:length].iloc[:,0:1]) for i in item]
        comp = pd.DataFrame()
        comp['original'] = tmp
        comp['predict'] = np.array(tempModel.predict(max_len-20,max_len-1))
        comp.plot()
        '''
        #数据并没有问题，因为一笔交易确实变不了多少，确实预测结果也就在0.01左右
        '''
        comp=pd.DataFrame()
        tmp=[i for item in np.array(data[length-5:length].iloc[:,0:1]) for i in item]
        comp['original'] = tmp
        predict=np.array(tempModel.predict(max_len-5,max_len-1))
        comp['predict']=predict
        comp.plot()
        '''
        predict=np.array(tempModel.predict(max_len,max_len+5))
        count=0
        '''
        for i in range(0,len(predict)):
            if predict[i]<np.array(data)[length-1]:
                count+=1
        if count>3:#下降序列
            comp=pd.DataFrame()
            comp['predict']=predict
            comp.plot()
            return -1
        if count<=1:
            return 1
        '''
    
        size=6
        a=[j for j in range(0,size)]
        b1=[[i] for item in np.array(data[length-size:length].iloc[:,0:1]) for i in item]
        c1=LinearRegression().fit(b1,a).coef_
        d1=list(c1)[0]
    
        b=[[x]for x in predict]
        c=LinearRegression().fit(b,a).coef_
        d=list(c)[0]
        if d>0.15 and d1<-0.15:
            vote+=1*weight_arma
        elif d<-0.1 and d1>0.1:
            vote+=-1*weight_arma
        else:
            pass
        #做出买卖判断的条件，这里要求同时满足
        if vote==6:
            return 1
        elif vote==-6:
            return -1
        else:
            return 0
    

In [ ]:
import pandas as pd
import numpy as np
from random import randint, random, shuffle
from matplotlib import pyplot as plt
from tqdm import trange
import queue
%matplotlib inline

stock = StockTrade()
agent = Agent(stock)

stock_data = pd.read_csv('Stock_Price.csv') # 历史股价，只有god_list中的Bot可以读取，请不要在策略中使用这部分信息
price_TWTR, price_KO, price_GM = stock_data.TWTR.values/10, stock_data.KO.values/10, stock_data.GM.values/10 # 为了减少Bot的数目对价格进行了十分之一处理
prices = [price_TWTR, price_KO, price_GM]
# 随机Bot
random_list = ['Alice', 'Bob', 'Carol', 'Dave', 'Eve', 'Francis', 'Grace', 'Hance', 'Isabella', 'Jason', 'Kate', 'Louis', 'Margret', 'Nathan']
# 贪心Bot
greedy_list = ['Olivia', 'Paul', 'Queen', 'Richard', 'Susan', 'Thomas']
# 神Bot，用来稳定价格和历史信息基本相同
god_list = ['Uma', 'Vivian', 'Winnie', 'Xander', 'Yasmine', 'Zach']
name_list = random_list + greedy_list + god_list

def random_order(name):
    for j, s_name in enumerate(['TWTR', 'KO', 'GM']):
        rand_type, rand_price = random(), randint(-3, 3)
        if rand_type > 0.5:
            agent.placeOrder(SellStockOrder(stock, name, s_name, agent.history_price()[j][-1] - 0.01 * rand_price))
        else:
            agent.placeOrder(BuyStockOrder(stock, name, s_name, agent.history_price()[j][-1] + 0.01 * rand_price))
            
def greedy_order(name):
    for j, s_name in enumerate(['TWTR', 'KO', 'GM']):
        rand_type, rand_price = random(), randint(-2, 5)
        if len(agent.history_price()[j]) >= 2:
            if agent.history_price()[j][-1] >= agent.history_price()[j][-2]:
                agent.placeOrder(SellStockOrder(stock, name, s_name, agent.history_price()[j][-1] - 0.01 * rand_price))
            else:
                agent.placeOrder(BuyStockOrder(stock, name, s_name, agent.history_price()[j][-1] + 0.01 * rand_price))

def god_order(name, i): # 神Bot一旦入场会按照历史价格交易三次，让股价靠近此时的历史真实值
    for j, s_name in enumerate(['TWTR', 'KO', 'GM']):
        rand_type, rand_price = random(), randint(0, 1)
        if agent.history_price()[j][-1] >= prices[j][i]:
            for _ in range(3):
                agent.placeOrder(SellStockOrder(stock, name, s_name, prices[j][i] - 0.01 * rand_price))
        else:
            for _ in range(3):
                agent.placeOrder(BuyStockOrder(stock, name, s_name, prices[j][i] + 0.01 * rand_price))

                
                
################以下是可编辑部分，请在策略部分加上必要的注释（理清代码思路也利于后面整理成大作业报告）################

history_money=[]
upper=[queue.PriorityQueue() for i in range(0,3)]
def time_strategy(parms='参数可以自己选择并传入'):
    
    #TODO 时间策略，选择第6（前1/4）进场，既能抓住机会，也能避免风险
    select_num=6
    return select_num # 决定今天应该在第几个进场，数字范围在0~25之间，若大于等于26则意味着这一天不进场
                
def your_order(name='You', left_days=300):
    if left_days==2000-687:
        return 
    if 'You' in agent.stock.money.keys():
        #print(agent.stock.money['You'])
        history_money.append(agent.stock.money['You'])
    #TODO 定价策略
    #TODO 然后是若干买入和卖出的挂单,以下仅为示例，实际数目和顺序都可以自行调整
    
    for j, s_name in enumerate(['TWTR', 'KO', 'GM']):
        limit=False
        rand_type, rand_price = random(), randint(0, 1)
        sell_price=agent.history_price()[j][-1] - 0.01 * rand_price
        buy_price=agent.history_price()[j][-1] + 0.01 * rand_price
        #print(agent.history_price()[j])
        '''
        具体见上面func函数，是算法核心
        '''
        #调用判断是否需要购买/卖出的核心函数
        judge=func(agent.history_price()[j])
        if judge==-1:
            agent.placeOrder(SellStockOrder(stock, 'You', s_name, sell_price))
            limit=True
        elif judge==1 and not limit:
            agent.placeOrder(BuyStockOrder(stock, 'You', s_name, buy_price))
            upper[j].put(buy_price)
        else:
            pass

        #continue
        
        limit=False
        rand_type, rand_price = random(), randint(0, 1)
        buy_price=agent.history_price()[j][-1] - 0.01 * rand_price
        sell_price=agent.history_price()[j][-1] + 0.01 * rand_price
        #见好即收，及时止损
        if not upper[j].empty():
            tmp=upper[j].get()
            win_or_loss= (tmp-agent.history_price()[j][-1])/tmp
            if win_or_loss>0.1:
                agent.placeOrder(SellStockOrder(stock, 'You', s_name, sell_price))
                continue
            elif win_or_loss<-0.1:
                agent.placeOrder(SellStockOrder(stock, 'You', s_name, sell_price))
                continue
            else: upper[j].put(tmp)
        '''
        if len(agent.history_price()[j])<20:
            continue
        else:
            count=0
            for i in range(2,16):
                if agent.history_price()[j][-1]<agent.history_price()[j][-i]:
                    count+=1
            if count>12:
                agent.placeOrder(BuyStockOrder(stock, name, s_name, buy_price))
                limit=True
            count=0
            for i in range(2,16):
                if agent.history_price()[j][-1]>agent.history_price()[j][-i]:
                    count+=1
            if count>12 and not limit :
                agent.placeOrder(SellStockOrder(stock,name,s_name,sell_price))
                '''
        #最后清零
        if left_days<=25:
            for t in range(0,3):
                agent.placeOrder(SellStockOrder(stock,name,s_name,sell_price))
    #agent.placeOrder(SellStockOrder(stock, 'You', '股票名称', '买单价格'))
    
def execute():
    #for i in trange(stock_data.shape[0]):
    for i in trange(0,2000):
        traded = 0
        np.random.shuffle(name_list) # 打乱所有Bot的进场顺序，请勿修改
        for num, name in enumerate(name_list):
            if not traded: # 如果这一天还没有交易过，利用时间策略判断是否需要进场；这部分可简可繁，直接设置每天自己都最后一个进场也完全OK
                selected_num = time_strategy('参数可以自己选择并传入')
                if selected_num <= num: # 进场时间已到
                    your_order('You', 2000-i)
                    traded=1
        
            
            # 以上是可编辑部分  
            
            
            if name in god_list:
                god_order(name, i)
            elif name in greedy_list:
                greedy_order(name)
            else:
                random_order(name)
    agent.close_market() # 闭市清空未成功交易的挂单
    lab=pd.DataFrame()
    lab['history_money']=history_money
    lab.plot()
    print('\nCurrent Stock', stock, sep='\n')
    print('\nCurrency', stock.show_money(), sep='\n')
    

In [ ]:
import warnings 
with warnings.catch_warnings(): #注意warning ignore必须放在这个with里面
    warnings.filterwarnings("ignore") 
    warnings.simplefilter('ignore')
    execute() # 让市场开始运作，观察并报告结果
# 你也可以先把上面block中的72~75行注释掉，观察一下bot们的活动

进行了10次测试，最终得到现金价值在18100~22000之间  
平均是19236
嗯，这个回报率属实不太行，仔细说起来也就是卖的位置价格不错，另外通过短线特征挣了点波动的小钱  
（不过也算尽力了，毕竟对金融了解有限x  
写这个算法本身还是挺有意思的，特别是对着曲线和模型去自己想特征并组合机器学习方法建模，然后修改一开始一直亏钱的bug，最后终于能跑了qwq   
附件里面的test.ipynb有对老师提到的相关拓展项目的实现和分析，也请帮忙看一下啦  
谢谢！